In [15]:
import sys
sys.path.insert(0, "/Users/artur/workspace/recipedb")

In [16]:
from recipedb.db import get_db
import pandas as pd
import pickle

In [17]:
db = get_db()

In [18]:
classes = pickle.load(open("classes.dump", "rb"))
annotations = pickle.load(open('annot.dump', 'rb'))

In [19]:
data = []
for classs in classes:
    data.append({
        "name": classes[classs],
        "class_id": classs
    })

In [20]:
cleaned_annotations = list(map(lambda x: { 'text': x['text'].lower(), 'class': list(map(lambda y: y['id'],x['class'])) }, annotations))

In [23]:
already_seen = set()
cleaned_annots = []
for annotation in cleaned_annotations:
    if annotation['text'] not in already_seen:
        already_seen.add(annotation['text'])
        cleaned_annots.append(annotation)

In [26]:
db.FoodClasses.insert_many(data)

In [27]:
db.FoodBase.insert_many(cleaned_annots)

In [28]:
base_from_db = list(db.FoodBase.find())

In [29]:
import re

word_pattern = re.compile(r'\w+')
food_terms = []
for item in base_from_db:
    terms = word_pattern.findall(item['text'])
    n = len(terms)
    for term in terms:
        food_terms.append({
            "term": term,
            "foodbase_id": item['_id'], 
            "n": n
        })
    

In [30]:
db.FoodTerms.insert_many(food_terms)